Importing packages and modules

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import json
import pandas as pd
from datetime import datetime,time,timedelta
import pickle

# Scraping the tables for data
First step is to gather all the URLs for the different seasons and competitions.

In [2]:
base_url = 'https://understat.com/league'
leagues = ['EPL','La_liga','Bundesliga','Serie_A','Ligue_1','RFPL'] 
seasons = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

# The entirety of this code can be looped over either all leagues or all seasons (or both !) for more data 

Retrieving the data from the URLs

In [3]:
url = base_url + '/' + leagues[0] + '/' + seasons[7]
res = requests.get(url)
soup = BeautifulSoup(res.content, 'lxml')

# finding the appropriate scripts
scripts = soup.find_all('script')

json_string = ''

for el in scripts:
    if 'teamsData' in str(el):
        json_string = str(el).strip()

# finding the start and end point of the data
json_string_start = json_string.index("('") + 2
json_string_end = json_string.index("')")

json_data = json_string[json_string_start:json_string_end]

json_data = json_data.encode('utf8').decode('unicode_escape')

## Putting data into pandas dataframes
Now that we have extracted the json data we can convert it into a python dictionary

In [4]:
#loading in the json data
data = json.loads(json_data)
teams = {}
#get teams and their relevant ids and put them into seperate dictionary
for id in data.keys():
    teams[id] = data[id]['title']
print(teams)


{'71': 'Aston Villa', '72': 'Everton', '74': 'Southampton', '75': 'Leicester', '78': 'Crystal Palace', '79': 'Norwich', '80': 'Chelsea', '81': 'West Ham', '82': 'Tottenham', '83': 'Arsenal', '86': 'Newcastle United', '87': 'Liverpool', '88': 'Manchester City', '89': 'Manchester United', '90': 'Watford', '92': 'Burnley', '220': 'Brighton', '229': 'Wolverhampton Wanderers', '244': 'Brentford', '245': 'Leeds'}


In [5]:
#list of column names
columns = []
#list of values from first datapoint
values = []
#taking the column headers and values of said columns from the the array of 'history' dictionaries for the first game for each team
for id in data.keys():
    columns = list(data[id]['history'][0].keys())
    values = list(data[id]['history'][0].values())
    break
#taking the value here can be used to verify the voracity of the data you collect.

In [6]:
#getting data for all the teams
list_of_dataframes_h = {}
list_of_dataframes_a = {}
for id, team in teams.items():
    teams_data = []
    for row in data[id]['history']:
        teams_data.append(list(row.values()))
    
    df = pd.DataFrame(teams_data,columns = columns)
    #this section of code is used to split data into home and away tables
    list_of_dataframes_h[team] = df.loc[df['h_a'] == 'h']
    list_of_dataframes_a[team] = df.loc[df['h_a'] == 'a']
    print(f"Added data for {team}")
    
    

Added data for Aston Villa
Added data for Everton
Added data for Southampton
Added data for Leicester
Added data for Crystal Palace
Added data for Norwich
Added data for Chelsea
Added data for West Ham
Added data for Tottenham
Added data for Arsenal
Added data for Newcastle United
Added data for Liverpool
Added data for Manchester City
Added data for Manchester United
Added data for Watford
Added data for Burnley
Added data for Brighton
Added data for Wolverhampton Wanderers
Added data for Brentford
Added data for Leeds


### Formatting the data
Now we quickly format the data into a more friendly format

In [7]:
for team,df in list_of_dataframes_h.items():
    list_of_dataframes_h[team]['ppda_coef'] = list_of_dataframes_h[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
    list_of_dataframes_h[team]['oppda_coef'] = list_of_dataframes_h[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
    
for team,df in list_of_dataframes_a.items():
    list_of_dataframes_a[team]['ppda_coef'] = list_of_dataframes_a[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
    list_of_dataframes_a[team]['oppda_coef'] = list_of_dataframes_a[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)

Now we can sum and take the average of the metrics to have a cumulative table for the entire season.

In [8]:
columns_to_sum = cols_to_sum = ['xG', 'xGA', 'npxG', 'npxGA', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'wins', 'draws', 'loses', 'pts', 'npxGD']
columns_to_avg = ['ppda_coef', 'oppda_coef']

In [9]:
frames_h = []
frames_a = []
for team, df in list_of_dataframes_h.items():
    scored_standard_deviation_h = pd.DataFrame.std(df['scored'])
    missed_standard_deviation_h = pd.DataFrame.std(df['missed'])
    sum_data = pd.DataFrame(df[columns_to_sum].sum()).transpose()
    avg_data = pd.DataFrame(df[columns_to_avg].mean()).transpose()
    final_df_h = sum_data.join(avg_data)
    final_df_h['matches'] = len(df)
    final_df_h['team'] = team
    final_df_h['G s.d'] = scored_standard_deviation_h
    final_df_h['GA s.d'] = missed_standard_deviation_h
    final_df_h['Gp90'] = final_df_h['scored']/final_df_h['matches']
    final_df_h['GAp90'] = final_df_h['missed']/final_df_h['matches']
    frames_h.append(final_df_h)

for team, df in list_of_dataframes_a.items():
    scored_standard_deviation_a = pd.DataFrame.std(df['scored'])
    missed_standard_deviation_a = pd.DataFrame.std(df['missed'])
    sum_data = pd.DataFrame(df[columns_to_sum].sum()).transpose()
    avg_data = pd.DataFrame(df[columns_to_avg].mean()).transpose()
    final_df_a = sum_data.join(avg_data)
    final_df_a['matches'] = len(df)
    final_df_a['team'] = team
    final_df_a['G s.d'] = scored_standard_deviation_a
    final_df_a['GA s.d'] = missed_standard_deviation_a
    final_df_a['Gp90'] = final_df_a['scored']/final_df_a['matches']
    final_df_a['GAp90'] = final_df_a['missed']/final_df_a['matches']
    frames_a.append(final_df_a)
  
full_stat_h = pd.concat(frames_h)
full_stat_a = pd.concat(frames_a)
full_stat_a.head()

,xG,xGA,npxG,npxGA,deep,deep_allowed,scored,missed,xpts,wins,...,pts,npxGD,ppda_coef,oppda_coef,matches,team,G s.d,GA s.d,Gp90,GAp90
0,10.599765,15.353355,9.838596,12.959095,79.0,74.0,10.0,16.0,10.9749,3.0,...,9.0,-3.120499,13.508110,8.480461,10,Aston Villa,0.816497,1.173788,1.000000,1.600000
0,10.859709,15.302181,9.337403,14.541008,44.0,79.0,9.0,18.0,11.4000,1.0,...,6.0,-5.203605,18.157827,8.261965,10,Everton,0.737865,1.032796,0.900000,1.800000
0,13.464214,20.372806,11.180704,19.611636,62.0,75.0,12.0,24.0,10.8284,2.0,...,9.0,-8.430932,10.715836,10.144366,11,Southampton,0.943880,1.250454,1.090909,2.181818
0,10.852216,17.093610,10.852216,14.048931,53.0,67.0,15.0,21.0,7.8840,2.0,...,9.0,-3.196715,14.394503,11.336097,9,Leicester,0.707107,1.658312,1.666667,2.333333
0,7.849294,16.699916,7.849294,15.177586,47.0,104.0,10.0,19.0,7.9914,1.0,...,7.0,-7.328292,22.225826,8.065464,10,Crystal Palace,1.154701,1.100505,1.000000,1.900000


Re-ordering columns for better readability

In [10]:
full_stat_h = full_stat_h[['team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'npxG', 'xGA', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts','G s.d','GA s.d','Gp90','GAp90']]
full_stat_h.sort_values('pts', ascending=False, inplace=True)
full_stat_h.reset_index(inplace=True, drop=True)
full_stat_h['position'] = range(1,len(full_stat_h)+1)

full_stat_a = full_stat_a[['team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'npxG', 'xGA', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts','G s.d','GA s.d','Gp90','GAp90']]
full_stat_a.sort_values('pts', ascending=False, inplace=True)
full_stat_a.reset_index(inplace=True, drop=True)
full_stat_a['position'] = range(1,len(full_stat_a)+1)

full_stat_h.head()

,team,matches,wins,draws,loses,scored,missed,pts,xG,npxG,...,ppda_coef,oppda_coef,deep,deep_allowed,xpts,G s.d,GA s.d,Gp90,GAp90,position
0,Manchester City,11,9.0,1.0,1.0,32.0,6.0,28.0,27.328953,25.045453,...,8.573525,32.053454,144.0,36.0,26.4759,2.467977,1.035725,2.909091,0.545455,1
1,Liverpool,10,7.0,3.0,0.0,25.0,6.0,24.0,26.027588,24.505248,...,7.731465,23.537951,107.0,43.0,24.7255,1.080123,0.843274,2.500000,0.600000,2
2,Tottenham,10,7.0,1.0,2.0,16.0,10.0,22.0,17.642363,17.642363,...,13.673521,11.866365,78.0,83.0,17.6344,1.074968,1.247219,1.600000,1.000000,3
3,Arsenal,10,7.0,1.0,2.0,18.0,8.0,22.0,20.075308,17.681047,...,13.437131,14.497240,89.0,51.0,20.6333,1.032796,0.918937,1.800000,0.800000,4
4,Chelsea,11,5.0,5.0,1.0,25.0,10.0,20.0,22.954730,19.910060,...,9.200478,19.290043,104.0,45.0,22.0991,1.902152,0.700649,2.272727,0.909091,5


Now we add xG difference into the dataframe

In [11]:
full_stat_h['xG_diff'] = full_stat_h['xG'] - full_stat_h['scored']
full_stat_h['xGA_diff'] = full_stat_h['xGA'] - full_stat_h['missed']
full_stat_h['xpts_diff'] = full_stat_h['xpts'] - full_stat_h['pts']

full_stat_a['xG_diff'] = full_stat_a['xG'] - full_stat_a['scored']
full_stat_a['xGA_diff'] = full_stat_a['xGA'] - full_stat_a['missed']
full_stat_a['xpts_diff'] = full_stat_a['xpts'] - full_stat_a['pts']

Now we convert to the appropriate datatypes for each column

In [12]:
columns_to_int = ['wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'deep', 'deep_allowed']
full_stat_h[columns_to_int] = full_stat_h[columns_to_int].astype(int)
full_stat_a[columns_to_int] = full_stat_a[columns_to_int].astype(int)

And now we make the final output cleaner

In [13]:
column_order = ['position','team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts', 'xG', 'xG_diff', 'npxG', 'xGA', 'xGA_diff', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts', 'xpts_diff','G s.d','GA s.d','Gp90','GAp90']
full_stat_h = full_stat_h[column_order]
full_stat_h.columns = ['#', 'Team', 'M', 'W', 'D', 'L', 'G', 'GA', 'PTS', 'xG', 'xG Diff', 'NPxG', 'xGA', 'xGA Diff', 'NPxGA', 'NPxGD', 'PPDA', 'OPPDA', 'DC', 'ODC', 'xPTS', 'xPTS diff','G s.d','GA s.d','Gp90','GAp90']
full_stat_a = full_stat_a[column_order]
full_stat_a.columns = ['#', 'Team', 'M', 'W', 'D', 'L', 'G', 'GA', 'PTS', 'xG', 'xG Diff', 'NPxG', 'xGA', 'xGA Diff', 'NPxGA', 'NPxGD', 'PPDA', 'OPPDA', 'DC', 'ODC', 'xPTS', 'xPTS diff','G s.d','GA s.d','Gp90','GAp90']

pd.options.display.float_format = '{:,.2f}'.format

In [14]:
full_stat_h.head()

,#,Team,M,W,D,L,G,GA,PTS,xG,...,PPDA,OPPDA,DC,ODC,xPTS,xPTS diff,G s.d,GA s.d,Gp90,GAp90
0,1,Manchester City,11,9,1,1,32,6,28,27.33,...,8.57,32.05,144,36,26.48,-1.52,2.47,1.04,2.91,0.55
1,2,Liverpool,10,7,3,0,25,6,24,26.03,...,7.73,23.54,107,43,24.73,0.73,1.08,0.84,2.50,0.60
2,3,Tottenham,10,7,1,2,16,10,22,17.64,...,13.67,11.87,78,83,17.63,-4.37,1.07,1.25,1.60,1.00
3,4,Arsenal,10,7,1,2,18,8,22,20.08,...,13.44,14.50,89,51,20.63,-1.37,1.03,0.92,1.80,0.80
4,5,Chelsea,11,5,5,1,25,10,20,22.95,...,9.20,19.29,104,45,22.10,2.10,1.90,0.70,2.27,0.91


In [15]:
full_stat_a.head()

,#,Team,M,W,D,L,G,GA,PTS,xG,...,PPDA,OPPDA,DC,ODC,xPTS,xPTS diff,G s.d,GA s.d,Gp90,GAp90
0,1,Manchester City,11,9,1,1,22,7,28,24.87,...,11.34,23.36,121,47,25.72,-2.28,1.26,0.67,2.00,0.64
1,2,Chelsea,11,7,2,2,20,7,23,18.30,...,11.66,15.75,115,53,19.61,-3.39,1.17,0.92,1.82,0.64
2,3,Liverpool,11,6,3,2,30,12,21,28.96,...,8.33,28.88,144,57,23.47,2.47,1.56,1.22,2.73,1.09
3,4,West Ham,11,6,2,3,19,12,20,17.44,...,11.69,15.64,59,72,16.76,-3.24,1.74,0.83,1.73,1.09
4,5,Wolverhampton Wanderers,10,5,2,3,9,7,17,10.44,...,19.61,9.66,47,74,12.17,-4.83,0.99,0.82,0.90,0.70


We can save this dataframe to be used for other projects. We can save it either as a csv or use the pickling module to save it. I chose to pickle it due to ease of use for further projects.

In [ ]:
full_stat_h.to_pickle("./EPL_table_h.pkl")

full_stat_a.to_pickle("./EPL_table_a.pkl") 

# Scraping Fixtures
You can use a similar method to scrape the fixtures for the next week from understat.

In [16]:
#this time we are looking for 'datesData' in te script
#clearing the json string
json_string = ''

for el in scripts:
    if 'datesData' in str(el):
        json_string = str(el).strip()

# again finding the start and end point of the data
json_string_start = json_string.index("('") + 2
json_string_end = json_string.index("')")

json_data = json_string[json_string_start:json_string_end]

json_data = json_data.encode('utf8').decode('unicode_escape')

data = json.loads(json_data)

Now we have the json fixtures data loaded we notice that it takes the form of a list of dictionaries with every fixture. To make sure we have all the fixtures for the season we will set the date after the end of all the leagues.

In [17]:
#bit of code to find the end of the leagues
leagues_end = datetime(2022,6,1)
leagues_end

datetime.datetime(2022, 6, 1, 0, 0)

Now we take all the fixtures between today and and the end of the week and store it into a dictionary and create a dataframe out of them.

In [19]:
fixtures_data = {'Home Team': [],'Away Team': [],'Home Team id': [],'Away Team id': []}
today = datetime.combine(datetime.now().date(),time(0,0))
for fixture in data:
    fixture_date = datetime.strptime(fixture['datetime'],'%Y-%m-%d %H:%M:%S')
    if fixture_date > today and fixture_date < leagues_end:
        fixtures_data["Home Team"].append(fixture['h']['title'])
        fixtures_data["Away Team"].append(fixture['a']['title'])
        fixtures_data["Home Team id"].append(fixture['h']['id'])
        fixtures_data["Away Team id"].append(fixture['a']['id'])
fixtures_df = pd.DataFrame(fixtures_data)
fixtures_df

,Home Team,Away Team,Home Team id,Away Team id
0,Watford,Norwich,90,79
1,Everton,Aston Villa,72,71
2,Arsenal,Burnley,83,92
3,Brentford,Wolverhampton Wanderers,244,229
4,Leeds,Newcastle United,245,86
...,...,...,...,...
155,Crystal Palace,Manchester United,78,89
156,Leicester,Southampton,75,74
157,Liverpool,Wolverhampton Wanderers,87,229
158,Manchester City,Aston Villa,88,71


This code can be tweaked to extract more data or to look for historical data. Again, we can dump this data into a .pickle format to make it easy to use in later projects.

In [ ]:
fixtures_df.to_pickle("./EPL_fixtures.pkl")